<a href="https://colab.research.google.com/github/rahiakela/mlops-research-and-practice/blob/main/practical-data-science-specialization/course-1-analyze-datasets-and-train-models-using-automl/register_and_visualize_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Register and visualize dataset

In this lab you will ingest and transform the customer product reviews dataset. Then you will use AWS data stack services such as AWS Glue and Amazon Athena for ingesting and querying the dataset. Finally you will use AWS Data Wrangler to analyze the dataset and plot some visuals extracting insights.

##Table of Contents

- [1. Ingest and transform the public dataset](#c1w1-1.)
  - [1.1. List the dataset files in the public S3 bucket](#c1w1-1.1.)
    - [Exercise 1](#c1w1-ex-1)
  - [1.2. Copy the data locally to the notebook](#c1w1-1.2.)
  - [1.3. Transform the data](#c1w1-1.3.)
  - [1.4 Write the data to a CSV file](#c1w1-1.4.)
- [2. Register the public dataset for querying and visualizing](#c1w1-2.)
  - [2.1. Register S3 dataset files as a table for querying](#c1w1-2.1.)
    - [Exercise 2](#c1w1-ex-2)
  - [2.2. Create default S3 bucket for Amazon Athena](#c1w1-2.2.)
- [3. Visualize data](#c1w1-3.)
  - [3.1. Preparation for data visualization](#c1w1-3.1.)
  - [3.2. How many reviews per sentiment?](#c1w1-3.2.)
    - [Exercise 3](#c1w1-ex-3)
  - [3.3. Which product categories are highest rated by average sentiment?](#c1w1-3.3.)
  - [3.4. Which product categories have the most reviews?](#c1w1-3.4.)
    - [Exercise 4](#c1w1-ex-4)
  - [3.5. What is the breakdown of sentiments per product category?](#c1w1-3.5.)
  - [3.6. Analyze the distribution of review word counts](#c1w1-3.6.)

##Setup

In [ ]:
!sudo pip -q install awscli --force-reinstall --upgrade
!aws --version

In [ ]:
!aws configure

Let's install the required modules first.

In [ ]:
# please ignore warning messages during the installation
!pip install --disable-pip-version-check -q sagemaker==2.35.0
!pip install --disable-pip-version-check -q awswrangler==2.7.0
#!pip install --disable-pip-version-check -q pandas==1.1.4
#!pip install --disable-pip-version-check -q numpy==1.18.5
#!pip install --disable-pip-version-check -q seaborn==0.11.0
#!pip install --disable-pip-version-check -q matplotlib===3.3.3

In [ ]:
# alternate dataset download
#!wget -q https://raw.githubusercontent.com/luongtruong77/practical-data-science/main/Course1-Analyze-Datasets-and-Train-ML-Models-using-AutoML/C1-W1/womens_clothing_ecommerce_reviews.csv

<a name='c1w1-1.'></a>
## 1. Ingest and transform the public dataset

The dataset [Women's Clothing Reviews](https://www.kaggle.com/nicapotato/womens-ecommerce-clothing-reviews) has been chosen as the main dataset.

It is shared in a public Amazon S3 bucket, and is available as a comma-separated value (CSV) text format:

`s3://dlai-practical-data-science/data/raw/womens_clothing_ecommerce_reviews.csv`

<a name='c1w1-1.1.'></a>
### 1.1. List the dataset files in the public S3 bucket

The [AWS Command Line Interface (CLI)](https://awscli.amazonaws.com/v2/documentation/api/latest/index.html) is a unified tool to manage your AWS services. With just one tool, you can control multiple AWS services from the command line and automate them through scripts. You will use it to list the dataset files.

**View dataset files in CSV format**

```aws s3 ls [bucket_name]``` function lists all objects in the S3 bucket. Let's use it to view the reviews data files in CSV format:

<a name='c1w1-ex-1'></a>
#### Exercise 1

View the list of the files available in the public bucket `s3://dlai-practical-data-science/data/raw/`.

**Instructions**:
Use `aws s3 ls [bucket_name]` function. To run the AWS CLI command from the notebook you will need to put an exclamation mark in front of it: `!aws`. You should see the data file `womens_clothing_ecommerce_reviews.csv` in the list.

In [4]:
!aws s3 ls s3://dlai-practical-data-science/data/raw/

# EXPECTED OUTPUT
# ... womens_clothing_ecommerce_reviews.csv

2021-04-30 02:21:06    8457214 womens_clothing_ecommerce_reviews.csv


<a name='c1w1-1.2.'></a>
### 1.2. Copy the data locally to the notebook

```aws s3 cp [bucket_name/file_name] [file_name]``` function copies the file from the S3 bucket into the local environment or into another S3 bucket. 

Let's use it to copy the file with the dataset locally.

In [5]:
!aws s3 cp s3://dlai-practical-data-science/data/raw/womens_clothing_ecommerce_reviews.csv ./womens_clothing_ecommerce_reviews.csv

download: s3://dlai-practical-data-science/data/raw/womens_clothing_ecommerce_reviews.csv to ./womens_clothing_ecommerce_reviews.csv


Now use the Pandas dataframe to load and preview the data.

In [6]:
import pandas as pd
import csv

df = pd.read_csv('./womens_clothing_ecommerce_reviews.csv', index_col=0)

df.shape

(23486, 10)

In [7]:
df.head()

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,847,33,"Cute, crisp shirt",If this product was in petite i would get the...,4,1,2,General,Tops,Blouses
1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,1049,50,My favorite buy!,I love love love this jumpsuit. it's fun fl...,5,1,0,General Petite,Bottoms,Pants
4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


<a name='c1w1-1.3.'></a>
### 1.3. Transform the data

To simplify the task, you will transform the data into a comma-separated value (CSV) file that contains only a `review_body`, `product_category`, and `sentiment` derived from the original data.

In [8]:
df_transformed = df.rename(columns={'Review Text': 'review_body',
                                    'Rating': 'star_rating',
                                    'Class Name': 'product_category'})
df_transformed.drop(columns=['Clothing ID', 'Age', 'Title', 'Recommended IND', 'Positive Feedback Count', 'Division Name', 'Department Name'], inplace=True)

df_transformed.dropna(inplace=True)

df_transformed.shape

(22628, 3)

In [9]:
df_transformed.head()

,review_body,star_rating,product_category
0,If this product was in petite i would get the...,4,Blouses
1,Love this dress! it's sooo pretty. i happene...,5,Dresses
2,I had such high hopes for this dress and reall...,3,Dresses
3,I love love love this jumpsuit. it's fun fl...,5,Pants
4,This shirt is very flattering to all due to th...,5,Blouses


Now convert the `star_rating` into the `sentiment` (positive, neutral, negative), which later on will be for the prediction.

In [10]:
def to_sentiment(star_rating):
  if star_rating in {1, 2}:  # negative
    return -1
  if star_rating == 3:       # neutral
    return 0
  if star_rating in {4, 5}:   # positive
    return 1

In [11]:
# transform star_rating into the sentiment
df_transformed["sentiment"] = df_transformed["star_rating"].apply(lambda star_rating: to_sentiment(star_rating))

In [12]:
df_transformed[:1]

,review_body,star_rating,product_category,sentiment
0,If this product was in petite i would get the...,4,Blouses,1


In [13]:
# so, now drop the star rating column
df_transformed.drop(columns=["star_rating"], inplace=True)

# and also remove reviews for product_categories with < 10 reviews
df_transformed = df_transformed.groupby("product_category").filter(lambda revies: len(revies) > 10)[["sentiment", "review_body", "product_category"]]
df_transformed.shape                 

(22626, 3)

In [14]:
df_transformed.head()

,sentiment,review_body,product_category
0,1,If this product was in petite i would get the...,Blouses
1,1,Love this dress! it's sooo pretty. i happene...,Dresses
2,0,I had such high hopes for this dress and reall...,Dresses
3,1,I love love love this jumpsuit. it's fun fl...,Pants
4,1,This shirt is very flattering to all due to th...,Blouses


<a name='c1w1-1.4.'></a>
### 1.4 Write the data to a CSV file

In [15]:
df_transformed.to_csv("./womens_clothing_ecommerce_reviews_transformed.csv", index=False)

In [16]:
!head -n 5 ./womens_clothing_ecommerce_reviews_transformed.csv

sentiment,review_body,product_category
1,If this product was in petite  i would get the petite. the regular is a little long on me but a tailor can do a simple fix on that.     fits nicely! i'm 5'4  130lb and pregnant so i bough t medium to grow into.     the tie can be front or back so provides for some nice flexibility on form fitting.,Blouses
1,"Love this dress!  it's sooo pretty.  i happened to find it in a store  and i'm glad i did bc i never would have ordered it online bc it's petite.  i bought a petite and am 5'8"".  i love the length on me- hits just a little below the knee.  would definitely be a true midi on someone who is truly petite.",Dresses
0,I had such high hopes for this dress and really wanted it to work for me. i initially ordered the petite small (my usual size) but i found this to be outrageously small. so small in fact that i could not zip it up! i reordered it in petite medium  which was just ok. overall  the top half was comfortable and fit nicely  but the bott

<a name='c1w1-2.'></a>
## 2. Register the public dataset for querying and visualizing

You will register the public dataset into an S3-backed database table so you can query and visualize our dataset at scale. 

<a name='c1w1-2.1.'></a>
### 2.1. Register S3 dataset files as a table for querying

Let's import required modules.

`boto3` is the AWS SDK for Python to create, configure, and manage AWS services, such as Amazon Elastic Compute Cloud (Amazon EC2) and Amazon Simple Storage Service (Amazon S3). The SDK provides an object-oriented API as well as low-level access to AWS services. 

`sagemaker` is the SageMaker Python SDK which provides several high-level abstractions for working with the Amazon SageMaker.

In [25]:
import boto3
import sagemaker
import pandas as pd
import numpy as np
import botocore

config = botocore.config.Config(user_agent_extra='dlai-pds/c1/w1', region_name="us-east-1")

# low-level service client of the boto3 session
sm = boto3.client(service_name='sagemaker', config=config)

sess = sagemaker.Session(sagemaker_client=sm)                         

bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = sess.boto_region_name
account_id = sess.account_id

print('S3 Bucket: {}'.format(bucket))
print('Region: {}'.format(region))
print('Account ID: {}'.format(account_id))

ValueError: ignored